### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2018 ###

# Making Miracles Happen Overnight: 24-Hour Shipping #

#### Evan Hernandez (ehernandez4@wisc.edu), Saman Tabatabai (tabatabai@wisc.edu), William Jen (wjen@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
    1. [Single Package, Single MOT](#2.A.-Single-Package,-Single-MOT)
    1. [Single Package, Many MOTs](#2.B.-Single-Package,-Many-MOTs)
    1. [Many Packages, Single MOT](#2.C.-Many-Packages,-Single-MOT)
    1. [Many Packages, Many MOTs](#2.D.-Many-Packages,-Many-MOTs)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Conclusion](#5.-Conclusion)
1. [References](#6.-References)

## 1. Introduction ##

At some point, you've probably needed to receive a package as soon as possible. Maybe you were driving to Niagara Falls from Boston, and realized at the Canadian border that you forgot your passport. Maybe your wedding was one day away, and you still needed an essential piece of decor. Or, if you were really unlucky, you were unconscious in a Des Moines ICU, and the medicine you needed was only available in Seattle. All of these unfortunate situations are remedied by overnight shipping services, thanks to companies like FedEx and UPS.

Today, people across the United States may request overnight package delivery for a nominal fee. As great as this is, one has to wonder: how can such a service be economically viable for shipping companies? The task is certainly feasible. For example, in the United States, a flight from Miami to Alaska requires only 12 hours, which leaves ample time for sorting and delivery. However, such flights may be exorbitantly expensive. Companies undoubtedly want to avoid sending each package on its own flight. More to the point, companies probably prefer to operate as few flights as possible on any given day.

Neither overnight shipping nor the challenges described above are new. FedEx began operating in April of 1973, and at the time delivered only 186 overnight packages using 14 cargo planes. Now, FedEx services nearly 10 million overnight packages per day, and is renowned for its innovative shipping technologies, which include the tracking number. FedEx offers overnight shipping from Seattle, WA, to Portland, ME; from Miami, FL, to Anchorage, AK; and even from Edinburgh, Scotland to anywhere in the United States.

Both UPS and FedEx operate a number of hubs. These hubs receive packages from nearby delivery points and transfer them to other hubs via daily cargo flights. For both companies, most packages run through the largest hub, located near the population mean of the United States in Memphis, TN and Louisville, KY, respectively. Flights carrying one day's packages arrive at the same time, plus or minus a few hours, every day. The packages are automatically sorted and then transferred to other hubs near their destination. Finally, they are delivered to their destination using cheaper methods of transportation, such as trucks and small propeller planes.

<img
    alt="US Population Mean"
    src="https://upload.wikimedia.org/wikipedia/commons/a/a0/US_Mean_Center_of_Population_1790-2010.PNG"
>

In the present report, we imagine we are a new shipping company with hubs in every city. Each hub connects to other hubs via different methods of transportation (trucks, flights, trains, etc.). We address the following question: how can we transfer packages between hubs to guarantee overnight delivery while minimizing transportation costs? 
Ultimately, we model overnight shipment planning as a network flow problem. We then compute an optimal shipping plan on synthetic data taken in part from the Google Maps API.

## 2. Mathematical model ##

We begin with a general construction of the problem. Suppose we wish to provide shipping services to $N$ cities, and we have access to $M$ methods of transportation (MOT). At the start of every day, we receive $P$ packages from arbitrary cities with arbitrary destinations. We guarantee delivery of each package from source to destination within $T$ hours. Our goal is to construct a feasible $T$-hour shipping plan that minimizes transportation cost.

Let the set of cities be denoted $V = \{v_1, \dots, v_N\}$, and let $e_{ij} \in V^2$ denote the edge between city $i$ and city $j$. To each such edge we attach a distance $d_{ij}$ in miles. For $k=1,\dots,M$ let the routes of the $k$-th MOT be given by the edge set $E_k \subseteq V^2$, and to each MOT associate an average speed $s_k$ in miles per hour, a fixed operating cost $f_k$, and a cost per unit of flow $c_k$. We also associate a multiplier $m_k$ to account for differences in distances between MOTs (e.g., trucks have more miles to cover than flights in general because they are restricted to roadways).

We make the following assumptions:
- There is a hub in each city.
- Shipping begins immediately at the start of the day.
- Once packages arrive in their destination city, they are delivered instantly after sorting.
- Sorting time is constant across hubs. We denote it to be $t$ hours.
- Transportation time scales linearly with distance and includes sorting time at the destination. In other words, if $e_{ij} \in E_k$, then the travel time for that edge is given by $d_{ij} s_k m_k + t$.
- Transportation cost scales linearly with capacity. In other words, if $e_{ij} \in E_k$ and $n$ packages flow over $e_{ij}$ for MOT $k$, then the cost is exactly $(n c_k + f_{k})m_kd_{ij}$.
- There is a feasible route between each pair of cities, i.e., a route whose travel time is less than $T$. This is a reasonable assumption in practice. For example, in the United States, you can fly between any two cities in less than 12 hours.
- Each MOT has unlimited capacity.

We will model and solve four variants of the overnight shipping problem, starting with the simplest instance. As we will see, the overnight shipping problem boils down to traditional variants of network flow. We will assign values to the variables in the [Solution](#3.-Solution) selection.

### 2.A. Single Package, Single MOT ###

We begin by considering overnight shipping where there is only one package to deliver, and only one MOT with which to deliver it. This amounts to a shortest path problem. Indeed, because there is only one MOT, we do not need to factor in the MOT travel times nor any costs, as they are evenly proportional to $d_{ij}$.


Let the package source be $u$ and the destination be $v$. Let $A \in \mathbb{R}^{|V| \times |E|}$ denote the incidence matrix and let $b \in \mathbb{R}^{|V|}$ be the vector with value 1 at $u$'s position, -1 at $v$'s position, and 0 everywhere else. (That is, $u$ must supply 1 unit of flow, and $v$ must demand 1 unit of flow.) Finally, let $x \in \mathbb{R}^{|E|}$ denote the flow along each edge and $d \in \mathbb{R}^{|E|}$ the distance associated with each edge. We obtain the following optimization problem:

\begin{align}
& \underset{x}{\text{minimize}} & d^\text{T}x \\
& \text{subject to} &  Ax = b \\
&& x \ge 0
\end{align}

There is one catch. We would like to treat the above as a linear program, but we require an integral solution. Thankfully, if we restrict the $d_{ij}$ to be integral, the incidence matrix for the graph $G = (V, E)$ will be totally unimodular, so we are guaranteed an integral solution to the LP. In practice, it is reasonable to round the $d_{ij}$ because they are on the order of hundreds, perhaps thousands of miles, and it is unlikely that the optimal shipping plan is sensitive to fractional changes in the distances.

### 2.B. Single Package, Many MOTs ###

We may readily extend (2a) to allow for multiple MOTs. It (almost) suffices to consider $E = \bigcup_{k=1}^M E_i$ and repeat the procedure from (2a) on the graph $(V, E)$. However, we must make two modifications.

First, travel time is now no longer evenly proportional to $d_{ij}$. Instead, transporting the package between cities $i$ and $j$ with MOT $k$ requires $d_{ij}m_ks_k + t$ hours. Let $\tau \in \mathbb{R}^{|E|}$ denote the vector of times required to travel across each edge. We constrain that $\tau^\text{T}x \le T$.

Second, and analogous to the above, cost is no longer evenly proportional to $d_{ij}$. Now, to each edge $e_{ijk} \in E_k$, we must associate the cost $(f_k + c_k)m_kd_{ij}$. If we group these costs into a vector $\alpha \in \mathbb{R}^{|E|}$, our new objective is to minimize $\alpha^\text{T}x$.  

Letting $A$, $b$, and $x$ be as before, the optimization problem becomes:

\begin{align}
& \underset{x}{\text{minimize}} & \alpha^\text{T}x \\
& \text{subject to} &  Ax = b \\
&& \tau^\text{T}x \le T \\
&& x \ge 0
\end{align}

Again, we treat the above as an LP and round the elements of $\alpha$ to guarantee an integral solution.

### 2.C. Many Packages, Single MOT ###

We model this as a <b>multiple commodity network flow (MCNF)</b> problem. In MCNF, we distinguish the flow of several commodities. Each node has a balance ledger for each commodity. Some nodes supply a commodity, while others demand it.
The objective is to ensure the demands are met while respecting edge capacities and minimizing flow costs.

In our case, the commodities are packages $p_i = (u_i, v_i)$ where $u_i$ is the source and $v_i$ is the sink. To each package we associate a separate decision variable $x_i \in \{0, 1\}^{|E|}$, which denotes the flow of the package, as well as a balance ledger $b_i$, defined in the same way as in (2a). For notational convenience, let $f, c, \tau \in \mathbb{R}^{|E|}$ denote the fixed cost, the cost per flow, and the travel time of each edge, respectively, and let $A$ be the incidence matrix for the graph as in (2a).

There are several issues to address. First, we need to ensure that all packages arrive within the deadline. We do this by constraining that $\tau^\text{T}x_i \le T$ for all $i$. Second, although we have only one MOT, we must consider the fixed cost of using each edge as well as the cost per flow. We introduce the binary vector $z \in \{0, 1\}^{|E|}$ to keep track of when an edge is used, and enforce the logical constraint $\sum_{i=1}^P x_i > 0 \Rightarrow z = 1$ using the big-$M$ trick on its contrapositive. We may then readily factor the fixed cost $f^\text{T}z$ into our objective function, in addition to the traditional flow cost $\sum_{i=1}^Pc^\text{T}x_i$.

We obtain the following optimization problem:

\begin{align}
& \underset{z, x_i}{\text{minimize}} & f^\text{T}z + \sum_{i=1}^P c^\text{T}x_i \\
& \text{subject to} & \sum_{i=1}^P x_i \le Pz \\ 
&& A x_i = b_i \text{ for } i=1,\dots,P \\
&& \tau^\text{T}x_i \le T \text{ for } i=1,\dots,P \\
&& x_i \in \{0,1\}^{|E|} \text{ for } i=1,\dots,P \\
&& z \in \{0,1\}^{|E|}
\end{align}

Unfortunately, MCF problems are significantly harder to solve than ordinary network flow problems, even with only two commodities. This is because we must constrain the $x_i$ (as well as $z$) to be binary, which elevates the original LP to an MIP.

### 2.D. Many Packages, Many MOTs ###

We apply the same modifications from (2b). The resulting optimization problem is the same as in (2c), but we redefine the edge set to be $E = \bigcup_{i=1}^M E_i$.

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [1]:
# this is a code block
using JuMP, Clp
m = Model(solver = ClpSolver())

things = [:horses, :donkeys, :goats]  # these are the things 
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses
solve(m)

for i in things
    println("The total number of ", i, " is: ", getvalue(x[i]))     # print result
end

The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0


Remember to make sure your code compiles! I will be running your code!

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.

## 6. References ##

- https://en.wikipedia.org/wiki/Mean_center_of_the_United_States_population
- https://upload.wikimedia.org/wikipedia/commons/a/a0/US_Mean_Center_of_Population_1790-2010.PNG
- https://about.van.fedex.com/our-story/history-timeline/history/
- https://www.wired.com/2013/04/40-years-of-fedex/
- https://www.youtube.com/watch?v=y3qfeoqErtY